# Part I: Introduction

## 1. 用Generative和Discriminative Model处理文本分类问题
**文本分类问题：** input X是sentence，label y是该sentence的类型标签。求$\hat{y}=\underset{y}{argmax}P(y|X)$ \
**关键：** 找到$P(y|X)$的分布形态 \
**方法：** \
· 概率方法：生成模型和判别模型（本节主要关注这两种方法的差异） \
· 非概率方法：SVM，Perceptron，NN不用softmax

### 1.1 生成模型和判别模型
**<font color=orange>Generative model</font>**
1. 模型训练时：用贝叶斯分析方式对$𝑃(𝑦|𝑋)$做分解，转化为对先验分布$P(y_i;φ)$和似然$P(X|y_i;θ)$建模，用MLE或者CE做目标函数，求解参数θ。\
贝叶斯分析：\
$P(y_i|X) = \frac{P(X|y_i)*P(y_i)}{P(X)} = \frac{P(X|y_i)*P(y_i)}{Σ_{y_j\in{Y}} P(X|y_j)*P(y_j)}$ \
求解MLE：\
$\hat{θ}=\underset{θ,φ}{argmax}\ L(θ,φ) = \underset{θ,φ}{argmax}\ \frac{P(X|y_i;θ)*P(y_i;φ)}{Σ_{y_j\in{Y}} P(X|y_j;θ)*P(y_j;φ)}$
2. 模型推理时：同样利用贝叶斯关系对目标函数做分析，求解优化函数。 \
利用贝叶斯分析： \
$\hat{y}=\underset{y}{argmax}P(y|X) = \underset{y}{argmax}\frac{P(X|y)*P(y)}{P(X)}$ \
<font color=red>由于y的取值与P(X)无关</font>，因此有：\
$\hat{y}=\underset{y}{argmax}\ P(X|y;\hat{θ})*P(y;\hat{φ})=\underset{y}{argmax}\ P(X,y;\hat{θ},\hat{φ})$

**<font color=orange>Discriminative model</font>**
1. 模型训练时：直接对$P(y|X)$的参数建模，用MLE或者CE做目标函数，求解参数θ:
$\hat{θ}=\underset{θ}{argmax}\ L(θ) = \underset{θ}{argmax}\ \underset{i}{Σ}logP(y_{i}|X_{i};θ)$
2. 模型推理时：直接用估计的条件概率分布求概率最大的类型： \
$\hat{y}=\underset{y}{argmax}P(y|X;\hat{θ})$

### 1.2 用简单的生成模型处理文本分类问题

#### 1.2.1 用生成模型解决问题的基础：语言模型
1. 生成模型在将后验概率转化为先验\*似然之后，实际应用场景下，比如分类问题中，一般会直接用训练集统计先验概率$P(y_i)$。然后分别对各个不同分类的样本集单独训练$P(X|y_i)$。从单个分类数据集而言，可以简写为$P(X)$
2. 在自然语言处理的语境中，X就是sentence。计算P(X)也就是计算sentence发生的概率。

<font color=blue>**语言模型：P(X)** </font>
1. **定义：models that assign probabilities to sequences of words.**
2. **Chain rule：**$P(X)= \prod_{i=1}^{I}P(x_i|x_1, x_2, ···,x_{i-1}) $  ，I是sentence长度
3. <font color=red>求解语言模型的关键是估计$P(x_i|x_1, x_2, ···,x_{i-1})$</font>
4. <font color=green>**最简单的近似估计方法是n-gram models：**</font>
$P(x_i|x_1, x_2, ···, x_{i-1}) \approx  P(x_i|x_{i-n+1}, x_{i-n+2}, ···, x_{i-1})$

#### 1.2.2 最简单的语言模型：count-based unigram model
**特点：**
1. naive bayesian假设：$P(x_i|x_1, x_2, ···, x_{i-1})=P(x_i)$ \
此时，得到的语言模型是unigram model：$P(X)= \prod_{i=1}^{I}P(x_i) $
2. 模型训练过程中，使用的求解$P(x_i)$的方法：MLE
3. 用MLE得到的结果是count-based estimate： 
$P_{MLE}(x_i)=\frac{C_{train}(x_i)}{Σ_{\tilde{x}}C_{train}(\tilde{x})}$

#### 1.2.3 BOW(Bag-of-words)生成模型: 又称为Naive Bayes
![BOW_generative_model](./pics/BOW_generative_model.png)
图中$θ_y$是在trainning set中计算的count-based probability。 \
用公式表达为：$P(X,y)=P(y)* {\textstyle \prod_{i=1}^{|X|}}P(x_i|y)=\frac{c(y)}{Σ_{y_j}c(y_j)}  {\textstyle \prod_{i=1}^{|X|}}\frac{c(x_i, y)}{Σ_{x_k}c(x_k, y)}$

#### 1.2.4 n-gram model
**特点：**
1. 假设：每个词出现的概率与前n-1个位置的词语有关。 \
$P(x_i|x_1, x_2, ···, x_{i-1})=P(x_i|x_{i-n+1}, x_{i-n+2}, ···, x_{i-1})$，简记为：$P(x_i|x_{i-n+1}:x_{i-1})$ 

2. 模型训练过程中，使用的求解$P(x_i|x_{i-n+1}:x_{i-1})$的方法：MLE
3. 用MLE得到的结果是count-based estimate： 
$P_{MLE}(x_i|x_{i-n+1}:x_{i-1})=\frac{C_{train}(x_{i-n+1}:x_i)}{Σ_{\tilde{x}}C_{train}(x_{i-n+1}:x_{i-1},\tilde{x})}=
\frac{C_{train}(x_{i-n+1}:x_i)}{C_{train}(x_{i-n+1}:x_{i-1})}$

**count table：** \
例：取n=2时，用Berkeley Restaurant Project的数据(|V|=1446)给8个words统计count table
<img src="./pics/BerkeleyRestaurantCountTable.png" style="zoom:80%">

将count table中的count转化为probability：
<img src="./pics/BerkeleyRestaurantProbTable.png" style="zoom:80%">

这种count table normalize之后拿到的probability可以反映语料的信息： \
(1) 语言本身的语义信息，比如：动词后面通常是名词或形容词  \
(2) 语言内容中体现的文化信息，比如：上表中，大比例的食物是中餐而不是西餐，说明语料涉及的人们更喜欢中餐

#### 1.2.5 N-gram model的问题：unknown words
**问题：** unknown words是指在training data中没有，却在Inference中出现的words。由于在trainning data中没有，直接用count-based MLE会得到count=0，$P(x_{unk})=0$，导致inference时出现：L(θ)=0

**处理思路：** 需要一种分布，对所有可能的words所赋予的概率都大于0.
1. 最常用的处理方法：character/subword-based model
2. smoothing，如，Laplace smoothing：实际上无效 \
· 思想：在训练模型时，给所有vocabulary的count都加1，因此unknown words的count=1。因此：\
$adjusted\ P(x_i)=\frac{C_{train}(x_i)+1}{Σ_{\tilde{x}}(C_{train}(\tilde{x})+1)} = \frac{C_{train}(x_i)+1}{Σ_{\tilde{x}}C_{train}(\tilde{x})+|V|}$ \
· 问题：unknown words的数量会很大，vocabulary要考虑所有unknown words的话，会导致$|V| \to \infty$,从而使$adjusted\ P(x_i)\to 0$
3. uniform distribution：基本不用 \
· 思想：在训练模型时，假设|V|固定，并让所有的unknown words的概率为$P_{unk}=\frac{1}{N_{|V|}}$

### 1.3 用判别模型解决文本分类问题的方法
判别模型的训练思路：$\hat{θ}=\underset{θ}{argmax}\ L(θ) = \underset{θ}{argmax}\  {\textstyle \sum_{i=1}^{|X|}}\ logP(y_{i}|X_{i};θ)
$

<font color=blue>generative classifier与discriminate classifier的简单比较:</font>
generative models的求解方式很绕，为了求P(y|X)要先求P(X|y),而disciminative model则直接求P(X|y)。但Distriminative model不能使用简单的count-based decomposition。

#### 1.3.1 BOW(Bag-of-words)判别模型
**二分类问题**：先计算score，再用logistic function转化为probability
1. 模型训练阶段：\
用MLE给每个word $x_i$找到$θ_{y|x_i}=P(y|x_i)$ \
这里minL(θ)用Gradient Descent
2. 模型推理阶段： \
先计算score：$score_{y|X}=θ_y+{\textstyle \prod_{i=1}^{|X|}}θ_{y|x_i} 
$ \
再转化为probability：$P(y|X;θ)=sigmoid(score_{y|X})=\frac{1}{1+e^{-score_{y|X}}}$

**多分类问题**：先计算score，再用softmax function转化为probability


## 2. 模型评估
### 2.1 extrinsic and intrinsic evaluation
1. 有两种评估类别： \
(1) extrinsic evaluation: 针对具体任务跑结果，做对比。就具体任务而言，这种方式的结果更准确，但是耗时，成本高。 \
(2) intrinsic evaluation: 思路是，在训练集上训练，然后在测试集上评估模型质量。因此可以独立于任务来对模型做评估，但是在具体任务上并不是100%准确。
2. 语言模型的intrinsic evaluation: **<font color=orange>probability and perplexity</font>**  \
(1) 如何合理的划分训练集和测试集：\
· 为了防止多次使用test set，通常将数据集分成trainning set, dev set和test set。实践中，通常将数据按照8:1:1来划分。 \
· 理论上：test set的大小至少要有足够的statistical power to measure a statistically significant difference between two models。 \
(2) **评价标准：** \
· **思路：**<font color=red>就两种语言模型的比较而言，在测试集上给出的probability更高的那个模型更好。</font>因为，给出的probability越高，意味着更好的预测了test set的出现是合理的。 \
· **metric**：<font color=blue>实践中通常不会直接用probability，而是用perplexity。后者对文本长度做了调整，而且有更好的可解释性。</font> \
(3) <font color=red>模型具有可比性的前提：训练时使用相同的vocabulary，测试时使用相同的test set。</font>

### 2.2 metric of intrinsic evaluation: perplexity(PPL)
1. 定义：X为test set word sequence，N=|X|，有：
$$\begin{eqnarray*}
PPL(X) & = & P(x_1, x_2, ..., x_n)^{-1/N}  ...①\\
 & = & [ {\textstyle \prod_{i=1}^{N}}\ P(x_i|x_1:x_{i-1}) ]^{-1/N} ...②
\end{eqnarray*}$$
2. 与probability的关系： \
$$\begin{align}
          \underset{θ}{argmin}\ PPL(X;θ) & = \underset{θ}{argmax}\  P(X;θ)^{1/N} \\
           & = \underset{θ}{argmax}\ \frac{1}{N}  \ {\textstyle \sum_{i=1}^{N}}\ logP(y_{i}|X_{i};θ)
          \end{align}$$ \
<font color=orange>**· 在模型训练时，MLE的优化目标是找到: $\hat{θ}=argmaxP(X;θ)$。此时，$P(X;θ)$的值是多少不重要。** </font>\
<font color=green>**· 而在模型评估时，则是将找到的以$\hat{θ}$为参数的模型用到test set上，看$P(X;θ)$的大小，用它来评价模型。** </font>
3. **理解：**


### 2.1 指标
1. Accuracy：$acc(y,\hat y)=\frac{1}{|y|}\sum_{i=1}^{|y|} Count(y_i=\hat{y}_i)$ \
<font color=red>问题：如果有出现概率非常小的类型，比如P(y=0)=1%，那么分类器只要全判y=1就有acc=99% </font>
2. precision（<font color=orange>**识别出来的目标的正确率**</font>）：估$\hat{y}=1$的样本中，确实是y=1的比例 \
$prec(y, \hat{y}) = \frac{c(y=1, \hat{y}=1)}{c(\hat{y}=1)} $
3. recall（<font color=orange>**有多少比例的目标被识别**</font>）：所有y=1的样本中，被估为$\hat{y}=1$的比例\
$rec(y, \hat{y}) = \frac{c(y=1, \hat{y}=1)}{c(y=1)} $
4. F-measure(F1 score)：$\frac{1*prec*rec}{prec+rec}$

### 2.2 统计检验(significant testing)的基本概念
1. p-value:
2. confidence interval

### 2.3 unpaired and paired test

### 2.4 bootstrap test